# Modeling #

## Import APIs ##

In [1]:
import torch
from torch import nn
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import wfdb
import ast

## Load data ##

### Metadata ###

In [2]:
ptbxl_df = pd.read_csv('./cleaned_data/cleaned_ptbxl_metadata.csv', index_col='ecg_id')

In [3]:
ptbxl_df.head()

,age,sex,device,validated_by_human,diagnostic_superclass,strat_fold,filename_lr,filename_hr
ecg_id,,,,,,,,
1,56.0,1,CS-12 E,True,['NORM'],3,records100/00000/00001_lr,records500/00000/00001_hr
2,19.0,0,CS-12 E,True,['NORM'],2,records100/00000/00002_lr,records500/00000/00002_hr
3,37.0,1,CS-12 E,True,['NORM'],5,records100/00000/00003_lr,records500/00000/00003_hr
4,24.0,0,CS-12 E,True,['NORM'],3,records100/00000/00004_lr,records500/00000/00004_hr
5,19.0,1,CS-12 E,True,['NORM'],4,records100/00000/00005_lr,records500/00000/00005_hr


In [4]:
metadata = ptbxl_df.loc[:, ['age', 'sex', 'device', 'validated_by_human']].copy()
metadata.head()

,age,sex,device,validated_by_human
ecg_id,,,,
1,56.0,1,CS-12 E,True
2,19.0,0,CS-12 E,True
3,37.0,1,CS-12 E,True
4,24.0,0,CS-12 E,True
5,19.0,1,CS-12 E,True


### Waveform data ###

In [5]:
waveform_data = []
for idx in ptbxl_df.index:
    record_path = ptbxl_df.loc[idx]['filename_hr']
    waveform_df = pd.read_csv('./cleaned_data/waveform_data/' + record_path + '.csv', index_col='Time (s)')
    waveform_data.append(waveform_df)
waveform_data = np.array(waveform_data)

In [6]:
waveform_data.shape

(21799, 1000, 12)

## Create recommended train-test split ##

This recommended train-test split code was obtained from the downloaded folder with the dataset: https://physionet.org/content/ptb-xl/1.0.3/.

In [7]:
# Split data into train and test
test_fold = 10

# Train
waveform_train = waveform_data[np.where(ptbxl_df.strat_fold != test_fold)]
metadata_train = metadata[ptbxl_df.strat_fold != test_fold]
y_train = ptbxl_df[ptbxl_df.strat_fold != test_fold].diagnostic_superclass

# Test
waveform_test = waveform_data[np.where(ptbxl_df.strat_fold == test_fold)]
metadata_test = metadata[ptbxl_df.strat_fold == test_fold]
y_test = ptbxl_df[ptbxl_df.strat_fold == test_fold].diagnostic_superclass

# Normalize data???

## Modeling ##

In [ ]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv1d()
        )
        self.decoder = nn.Sequential(
            
        )
    
    def forward(self, x):
        